In [1]:
from langchain_community.tools import tool
from dotenv import load_dotenv
import os
import requests

load_dotenv()

True

In [6]:
# create the tools to fetch weather and search some query
from langchain_community.tools import DuckDuckGoSearchRun

@tool
def get_weather(city:str):
    """Returns the weather condition of a particular city 
    given as input"""
    
    # take the url
    url = f"http://api.weatherstack.com/current?access_key={os.environ["WEATHER_STACK_API"]}&query={city}"
    
    # send a request to the API and get the result in JSON
    response = requests.get(url=url).json()
    
    # return the temperature and the condition in the city
    return (
        response["current"]["temperature"],
        response["current"]["weather_descriptions"][0]
    )
    
@tool
def search_query(query:str)->str:
    """Take a search query and returns 
    the response to that query after browing the internet"""
    
    search_tool  = DuckDuckGoSearchRun()
    
    response = search_tool.invoke(query)
    
    return response

In [7]:
# define an llm, a toolkit and a prompt to build the agent
from langchain_groq import ChatGroq
from langchain_classic.hub import pull

# llm
llm = ChatGroq(model="llama-3.3-70b-versatile")

# toolkit
toolkit = [search_query,get_weather]

# download prompt from langchain_hub
prompt = pull("hwchase17/react")

In [8]:
# define the agent now
from langchain_classic.agents import AgentExecutor,create_react_agent

agent = create_react_agent(
    llm=llm,
    tools=toolkit,
    prompt=prompt
)

# and the AgentExecutor
agent_executor = AgentExecutor(
    agent=agent,
    tools=toolkit,
    verbose=True
)

In [10]:
# test the agent
response = agent_executor.invoke({"input":"What is the current weather in Mumbai, and based on that suggest me good places to visit there"})
print(response)



> Entering new AgentExecutor chain...
Thought: To answer this question, I first need to find out the current weather in Mumbai. Once I have that information, I can suggest places to visit based on the weather conditions.

Action: get_weather
Action Input: Mumbai(25, 'Haze')Thought: Now that I know the current weather in Mumbai is Haze with a temperature of 25, I can suggest places to visit based on this information. Since it's hazy, it might be a good idea to visit indoor locations or places with minimal outdoor exposure.

Action: search_query
Action Input: places to visit in Mumbai on a hazy dayNovember 7, 2025 - This one-day Mumbai itinerary blends culture, history, shopping, and food experiences for a perfect city ... Start your morning with the calm sunrise at Juhu Beach, followed by a soulful visit to the Siddhivinayak Temple . 1 week ago - 💜 ... This itinerary is quite packed, so I recommend leaving early in the morning. It’s good to know that #3 the Elephanta Caves, #6 the Pri